<h1 style="text-align: center; color: RED">ARMUT DATA SETİ İLE ASSOCIATION RULE BASED RECOMMENDER SYSTEM
</h1>

<span style="color: blue; font-weight: bold;">1-İŞ PROBLEMİ</span>

Türkiye’nin en büyük online hizmet platformu olan Armut, hizmet verenler ile hizmet almak isteyenleri buluşturmaktadır.
Bilgisayarın veya akıllı telefonunun üzerinden birkaç dokunuşla temizlik, tadilat, nakliyat gibi hizmetlere kolayca
ulaşılmasını sağlamaktadır.
Hizmet alan kullanıcıları ve bu kullanıcıların almış oldukları servis ve kategorileri içeren veri setini kullanarak Association
Rule Learning ile ürün tavsiye sistemi oluşturulmak istenmektedir.

<span style="color: blue; font-weight: bold;">2-VERİ SETİ HİKAYESİ</span>

Veri seti müşterilerin aldıkları servislerden ve bu servislerin kategorilerinden oluşmaktadır. Alınan her hizmetin tarih ve saat
bilgisini içermektedir.

UserId :Müşteri numarası

ServiceId :Her kategoriye ait anonimleştirilmiş servislerdir. (Örnek : Temizlik kategorisi altında koltuk yıkama servisi).Bir ServiceId farklı kategoriler altında bulanabilir ve farklı kategoriler altında farklı servisleri ifade eder.
(Örnek: CategoryId’si 7 ServiceId’si 4 olan hizmet petek temizliği iken CategoryId’si 2 ServiceId’si 4 olan hizmet mobilya montaj)

CategoryId :Anonimleştirilmiş kategorilerdir.

CreateDate:Hizmetin satın alındığı tarih

| Sütun        | Açıklama                                                        |
| ------------ | --------------------------------------------------------------- |
| `UserId`     | Sepeti oluşturan kişi                                           |
| `ServiceId`  | Alınan hizmet ID                                                |
| `CategoryId` | Hizmetin ait olduğu kategori                                    |
| `CreateDate` | Sepet oluşturulma zamanı                                        |
| `Hizmet`     | ServiceId\_CategoryId birleştirilmiş hali (özgün hizmet tanımı) |
| `New_Date`   | Yıl-Ay formatında                                               |
| `SepetID`    | Her kullanıcı ve ay için oluşturulmuş sepet kimliği             |


<h2 style="text-align: center; color: RED">PROJE GÖREVLERİ
</h2>

<span style="color: blue; font-weight: bold;">GÖREV-1: Veriyi Hazırlama</span>

**Adım 1:** armut_data.csv dosyasını okutunuz.

**Adım 2:** ServisID her bir CategoryID özelinde farklı bir hizmeti temsil etmektedir. ServiceID ve CategoryID’yi "_" ile birleştirerek bu hizmetleri
temsil edecek yeni bir değişken oluşturunuz. Elde edilmesi gereken çıktı:

In [88]:
import pandas as pd
import datetime as dt
from mlxtend.frequent_patterns import apriori, association_rules

In [70]:
df = pd.read_csv("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\5. Recommendation Systems\\datasets\\armut_data.csv")

In [9]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   UserId      162523 non-null  int64 
 1   ServiceId   162523 non-null  int64 
 2   CategoryId  162523 non-null  int64 
 3   CreateDate  162523 non-null  object
dtypes: int64(3), object(1)
memory usage: 5.0+ MB


In [72]:
df['Hizmet'] = df['ServiceId'].astype(str) + '_' +  df['CategoryId'].astype(str)

In [ ]:
#alternatif çözüm
[str(row[0]) + '_' + str(row[1]) for row in df.values]

df.head()

**Adım 3:** Veri seti hizmetlerin alındığı tarih ve saatten oluşmaktadır, herhangi bir sepet tanımı (fatura vb. ) bulunmamaktadır. Association Rule
Learning uygulayabilmek için bir sepet (fatura vb.) tanımı oluşturulması gerekmektedir. Burada sepet tanımı her bir müşterinin aylık aldığı
hizmetlerdir. Örneğin; 7256 id'li müşteri 2017'in 8.ayında aldığı 9_4, 46_4 hizmetleri bir sepeti; 2017’in 10.ayında aldığı 9_4, 38_4 hizmetleri
başka bir sepeti ifade etmektedir. Sepetleri unique bir ID ile tanımlanması gerekmektedir. Bunun için öncelikle sadece yıl ve ay içeren yeni bir
date değişkeni oluşturunuz. UserID ve yeni oluşturduğunuz date değişkenini "_" ile birleştirirek ID adında yeni bir değişkene atayınız.
Elde edilmesi gereken çıktı:

In [74]:
df['CreateDate'] = pd.to_datetime(df['CreateDate'])
                                

In [62]:
df['New_Date'] = (df['CreateDate'].dt.year.astype(str)) + '-' + (df['CreateDate'].dt.month.astype(str))

In [76]:
#alternatif kod
df['New_Date'] = df['CreateDate'].dt.strftime('%Y-%m')

In [78]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08


In [80]:
df['SepetID'] = df['UserId'].astype(str) + '_' + df['New_Date'].astype(str)

In [82]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date,SepetID
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08


<span style="color: blue; font-weight: bold;">GÖREV-2: Birliktelik Kuralları Üretiniz ve Öneride bulununuz</span> 

**Adım 1:** Aşağıdaki gibi sepet, hizmet pivot table’i oluşturunuz.

In [110]:
pivot_df

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,...,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetID,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99_2017-12,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99_2018-01,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99_2018-02,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
pivot_df = df.pivot_table(
    index='SepetID',
    columns='Hizmet',
    values='UserId',
    aggfunc='count',
    fill_value=0
)


In [119]:
pivot_df = pivot_df.applymap(lambda x: 1 if x>0 else 0)

C:\Users\ASLI\AppData\Local\Temp\ipykernel_10204\1084866055.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_df = pivot_df.applymap(lambda x: 1 if x>0 else 0)


In [121]:
pivot_df

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,...,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetID,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99_2017-12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99_2018-01,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99_2018-02,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#alternatif yöntem
new_df = df_.groupby(['SepetId', 'Hizmet'])['Hizmet'].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

**Adım 2:** Birliktelik kurallarını oluşturunuz.

In [123]:
apriori_df = apriori(pivot_df,
                     min_support = 0.01,
                     use_colnames=True,
                     low_memory=True)

C:\Users\ASLI\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [125]:
rules = association_rules(apriori_df,
                         metric = 'support',
                         min_threshold=0.01)

In [127]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,1.0,0.005442,1.046325,0.488074,0.073635,0.044274,0.162388
1,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,1.0,0.005442,1.124216,0.449965,0.073635,0.110491,0.162388
2,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,1.0,0.018191,1.209066,0.609539,0.156242,0.172915,0.270631
3,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,1.0,0.018191,1.188833,0.616073,0.156242,0.158839,0.270631
4,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,1.0,0.007929,1.072262,0.803047,0.081967,0.067392,0.252086
5,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,1.0,0.007929,1.493211,0.725728,0.081967,0.330302,0.252086
6,(15_1),(38_4),0.120963,0.066568,0.011177,0.092397,1.388001,1.0,0.003124,1.028458,0.318007,0.063376,0.027671,0.130147
7,(38_4),(15_1),0.066568,0.120963,0.011177,0.167897,1.388001,1.0,0.003124,1.056404,0.299475,0.063376,0.053392,0.130147
8,(15_1),(49_1),0.120963,0.067762,0.010011,0.082763,1.221375,1.0,0.001815,1.016354,0.206192,0.056018,0.016091,0.115252
9,(49_1),(15_1),0.067762,0.120963,0.010011,0.147741,1.221375,1.0,0.001815,1.031420,0.194425,0.056018,0.030463,0.115252


**Adım 3:** arl_recommender fonksiyonunu kullanarak en son 2_0 hizmetini alan bir kullanıcıya hizmet önerisinde bulununuz.

In [131]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]


In [141]:
arl_recommender(rules, '2_0', 3)

['22_0', '25_0', '15_1']